In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import load_data as ld
import seaborn as sns
import geopandas as gpd

dir_data = "C:\\Users\\Nicolas\\Data\\data.gouv.fr\\"

# Véhicules immatriculés impliqués dans les accidents corporels - 2009 à 2021
dic_vehic_immat = ld.load_vehic_immat_gouv(folder_path=dir_data, start_year=2009, end_year=2021)

# Caractéristiques
dic_caract = ld.load_caract_gouv(folder_path=dir_data, start_year=2005, end_year=2021)

# Lieux
dic_lieux = ld.load_lieux_gouv(folder_path=dir_data, start_year=2005, end_year=2021)

# Usagers
dic_usagers = ld.load_usagers_gouv(folder_path=dir_data, start_year=2005, end_year=2021)

# Véhicules
dic_vehic = ld.load_vehicules_gouv(folder_path=dir_data, start_year=2005, end_year=2021)


In [ ]:
# Etude des fichiers usagers-yyyy.csv

# 1 : Indemne
# 2 : Tué
# 3 : Blessé hospitalisé
# 4 : Blessé léger

print("Usagers - Dimension des données selon les années : \n")
for year in range(2018, 2022):
    print(f"usagers {year} : {dic_usagers[year].shape}")

print("\nUsagers - Variables selon les années : \n")
for year in range(2018, 2022):
    print(f"usagers {year} : {list(dic_usagers[year].columns)}")

print("\nUsagers - Modalités de la variable grav : \n")
for year in range(2018, 2022):
    print(f"usagers {year} : {dic_usagers[year]['grav'].value_counts()}")


plt.figure(figsize=(16, 16))
plt.title('Répartition de la gravité des accidents par année depuis 2005')
for k in range(1, 18):
    year = 2005 + k -1
    plt.subplot(5, 4, k)
    plt.hist(dic_usagers[year].grav);
    plt.title(year)

## Analyse usagers

In [ ]:
# Analyse rapide des colonnes du fichier usager

help(ld.load_usagers_gouv)

df = dic_usagers[2020]
print(list(df.columns))

fig = plt.figure(figsize=(12, 12))
ax = fig.gca()
df[['place', 'catu', 'grav', 'sexe', 'an_nais', 'trajet', 'secu1', 'secu2', 'locp', 'actp', 'etatp']].hist(ax = ax);
# conducteur très majoritairement impliqué
# ~70% d'hommes vs ~30% femmes
# pic de jeunes conducteurs
# pics sur promenade et trajets domicile-travail

## Analyse caracteristiques

In [ ]:
# Analyse rapide des colonnes du fichier caractéristiques

help(ld.load_caract_gouv)

df = dic_caract[2020]
print(list(df.columns))

df.head()
# cols = ['mois', 'hrmn', 'lum', 'dep', 'com', 'agg', 'int', 'atm', 'col', 'lat', 'long']
fig = plt.figure(figsize=(10, 10))
ax = fig.gca()
df[['mois', 'lum', 'dep', 'com', 'agg', 'int', 'atm', 'col']].hist(ax = ax);


## Analyse vehicules

In [ ]:
# Analyse rapide des colonnes du fichier vehicules

help(ld.load_vehicules_gouv)

df = dic_vehic[2020]
print(list(df.columns))

df.head()
# cols = ['mois', 'hrmn', 'lum', 'dep', 'com', 'agg', 'int', 'atm', 'col', 'lat', 'long']
fig = plt.figure(figsize=(10, 10))
ax = fig.gca()
df[['senc', 'catv', 'obs', 'obsm', 'choc', 'manv', 'motor', 'occutc']].hist(ax = ax);


## Analyse lieux

In [ ]:
# Analyse rapide des colonnes du fichier vehicules

help(ld.load_lieux_gouv)

df = dic_lieux[2020]
print(list(df.columns))

df.head()
fig = plt.figure(figsize=(10, 10))
ax = fig.gca()
df[['catr', 'circ', 'nbv', 'vosp', 'prof', 'plan', 'surf', 'infra', 'situ', 'vma']].hist(ax = ax);


## Analyse véhicules immat impliqués

In [ ]:
# Analyse rapide des colonnes du fichier dés véhicules immatriculés impliqués

help(ld.load_vehic_immat_gouv)

df = dic_vehic_immat[2020]
print(list(df.columns))

df.head()
fig = plt.figure(figsize=(10, 10))
ax = fig.gca()
df[['place', 'catu', 'grav', 'sexe', 'an_nais', 'trajet', 'secu1', 'secu2', 'secu3', 'locp', 'actp', 'etatp']].hist(ax = ax);


## Merge de toutes les données

In [ ]:
df_caract = dic_caract[2021]
df_lieux = dic_lieux[2021]
df_usagers = dic_usagers[2021]
df_vehic = dic_vehic[2021]
df_vehic_immat = dic_vehic_immat[2021] 

# print('caract :', list(df_caract.columns))
# print('lieux :', list(df_lieux.columns))
# print('usagers :', list(df_usagers.columns))
# print('vehic :', list(df_vehic.columns))
# print('vehic_immat :', list(df_vehic_immat.columns))


# pas de doublons de Num_Acc dans df_caract, et df_lieux -> on peut merger df_usagers, df_caract, et df_lieux sur Numm_Acc
df_caract.Num_Acc.duplicated().sum() + df_lieux.Num_Acc.duplicated().sum()


usagers_caract = df_usagers.merge(right=df_caract, on='Num_Acc', how='left')
usagers_lieux  = df_usagers.merge(right=df_lieux,  on='Num_Acc', how='left')

# cor = usagers_caract.corr()
cor = usagers_lieux.corr()

fig, ax = plt.subplots(figsize=(12,12))
sns.heatmap(cor, annot=True, ax=ax, cmap='coolwarm');




## Geolocation

In [ ]:
# convert latitude and longitude to good format
df_caract['lat'] = df_caract['lat'].str.replace(',', '.')
df_caract['long'] = df_caract['long'].str.replace(',', '.')
df_caract_33 = df_caract[df_caract.dep == '33']
df_caract_33 = df_caract_33.iloc[0:10, :]

# print(df_caract_min.head())

gdf = gpd.GeoDataFrame(df_caract_33, geometry=gpd.points_from_xy(df_caract_33.long, df_caract_33.lat))
plt.figure(figsize=(20, 20))
gdf.plot(column='lum')

# 'lum'
# Lumière : conditions d’éclairage dans lesquelles l'accident s'est produit :
# 1 – Plein jour
# 2 – Crépuscule ou aube
# 3 – Nuit sans éclairage public
# 4 – Nuit avec éclairage public non allum

plt.figure(figsize=(20, 20))
gdf.explore()
# gdf.explore(column='lum')
# deps_simple = gpd.read_file('.\\json-maps\\departements-version-simplifiee.geojson')
# deps = gpd.read_file('.\\json-maps\\departements.geojson')

# plt.figure(figsize=(20, 20))
# display Gironde
# deps.loc[33].geometry

# display carte de france découpée par départements
# deps.plot(figsize=(10,10), linewidth=2)

# grille associée
# deps.crs
# import contextily as cx
# cx.providers.keys()

